### **5.10 - [Practica] Sending tasks to a specific worker with Queues**

Cuando usamos el CeleryExecutor, podemos especificar la cola (el queue) donde queremos enviar algunas de las tareas de un DAG dado.  Por ejemplo, digamos que tienes una tarea muy intensiva en CPU y uno de tus nodos worker tiene más CPUs que los otros. Puede ser inteligente enviarla a este worker.  ¿Cómo? Mediante el uso de las Celery queues. Otro ejemplo podría ser una tarea que interactúa con Spark y sólo un worker tiene Spark instalado en tu cluster.  En ese caso, tendrías que enviar esta tarea al worker correspondiente.  Básicamente, puedes especializar tus workers tanto desde el punto de vista de los recursos como desde el punto de vista del entorno.  

<center><img src="https://i.postimg.cc/dV4Qb3mz/a879.png"></center>

Para utilizar una cola hay que hacer dos cosas:  En primer lugar, cuando crees un nuevo worker con el comando "airflow worker", tienes que añadir la opción **`-q`** con el nombre de la cola para que el worker sólo recoja tareas cableadas a esa cola especificada.  Ten en cuenta que puedes añadir varias colas separándolas con comas.  Lo último que hay que hacer es establecer el atributo "**`queue`**" de la tarea que se quiere enviar. Todos los Operadores tienen un atributo llamado "**`queue`**".  Por defecto está establecido en "**`default`**".  Si vas a tu editor de código, abre el archivo airflow.cfg de mnt/airflow y busca el parámetro llamado "**`default_queue`**".  Puedes ver que la cola por defecto donde las tareas se asignan y los workers escuchan es "**`default`**".  

<center><img src="https://i.postimg.cc/65K9TnMk/a878.png"></center>

Así que, sin más espera vamos a crear nuestras primeras colas. Abre el archivo docker-compose-CeleryExecutor.yml. A continuación, en la parte inferior del archivo, tenemos el **`service worker`**. 

<center><img src="https://i.postimg.cc/65MDwvgL/a880.png"></center>

En la instrucción del comando, después de "worker" añade **`"-q worker_ssd,worker_cpu ,worker_spark"`**.  Básicamente, haciendo esto, vamos a crear tres colas respectivamente llamadas worker_ssd, worker_cpu y worker_spark, de acuerdo a la especificidad de cada worker.  Imaginemos que worker_1 tiene un ssd para tareas intensivas de I/O, entonces escuchará a la cola worker_ssd. El worker_2 tiene más CPUs que los otros workers y por lo tanto se adaptará mejor para procesar tareas intensivas de CPU.  Por lo tanto, lo adjuntaremos a la cola worker_cpu. Finalmente, el último worker, worker_3, está ubicado en un cluster de Spark, por lo que recibirá tareas de la cola worker_spark.  

<center><img src="https://i.postimg.cc/XYXtvv0M/a881.png"></center>

Por supuesto, como estamos usando contenedores docker en la misma máquina, no es cierto, pero en realidad, es lo que podrías hacer en tu empresa. Por lo tanto, guardar el archivo y vuelve a la terminal. Ahora, inicia los contenedores docker escribiendo **`"docker-compose -f docker-compose-CeleryExecutor.yml up -d --scale worker=3"`**.  Enter.  

<center><img src="https://i.postimg.cc/m2L00wXg/a882.png"></center>

Muy bien, ahora vamos a abrir el Flower dashboard.  

<center><img src="https://i.postimg.cc/C5YWMHwJ/a883.png"></center>

Así que tenemos tres nodos worker como se muestra aquí, pero la parte interesante es que, si hacemos clic en uno de ellos y hacemos clic en "Queues", obtenemos las tres colas que hemos creado anteriormente desde el archivo docker-compose.  

<center><img src="https://i.postimg.cc/FR0MQnX7/a884.png"></center>
<center><img src="https://i.postimg.cc/j2XBDw5T/a885.png"></center>

Bien, vamos a ejecutar un DAG desde la interfaz de Airflow para comprobar si las colas funcionan.  Ok, vamos a ejecutar un DAG desde la interfaz de usuario de Airflow para probar las colas. Bien, DAGRun está corriendo. 

<center><img src="https://i.postimg.cc/Wp6BWK7y/a886.png"></center>

Revisemos que las tareas están siendo procesadas por los nodos worker. En el Flower dashboard, haz click en "Dashboard".  Ok, creo que tenemos un problema.  

<center><img src="https://i.postimg.cc/xjRZymfD/a887.png"></center>

No se está procesando ninguna tarea.  Si recuerdas, te dije antes que por defecto las tareas se extraen de la cola llamada "default" definida por el parámetro "default_queue" en airflow.cfg.  

<center><img src="https://i.postimg.cc/htHZPbrx/a888.png"></center>

Dado que la cola ya no existe, no hay manera de que los workers extraigan las tareas ni de que el scheduler las envíe.  Así que vamos a arreglar esto.  Abre tu terminal y detén los contenedores docker escribiendo "docker-compose -f docker-compose-CeleryExecutor.yml down".  A continuacion, en el archivo docker-compose, vamos a añadir la cola por defecto (default queue) que será la cola donde se enviarán las tareas sin requisitos especiales. De vuelta al editor de código, añadimos "default" al final del comando, así. Guardamos el archivo.  

<center><img src="https://i.postimg.cc/9fZ84R8k/a889.png"></center>

Ahora, antes de iniciar el cluster, tenemos que especificar la cola que queremos para cada tarea que tenga requisitos específicos.  Abre el archivo queue_dag.py en la carpeta mnt/airflow. Este DAG es bastante simple.  Ejecuta 7 tareas donde 6 de ellas se pueden ejecutar en paralelo. La tarea 7 depende de ellas. Además, cada tarea tiene un sufijo, _io, _cpu, _spark para indicar la cola a la que se debe enviar una tarea determinada.  

<center><img src="https://i.postimg.cc/NMtNRsZj/a890.png"></center>

Entonces, ¿cómo podemos enviar cada tarea a la cola adecuada para que la ejecute el worker correcto? Pues bien, sólo tenemos que añadir el argumento "queue" al operador de tarea. Por ejemplo, para la tarea t_1_ssd, escribimos "queue="worker_ssd"".  

<center><img src="https://i.postimg.cc/SNxvZKRX/a891.png"></center>

Haciendo esto, el scheduler enviará esa tarea a la cola "worker_ssd" mientras que el worker que escucha esa cola tirará de la tarea para ejecutarla.  Bien, ahora como ejercicio, tienes que asignar la cola "worker_ssd", "worker_cpu" o "worker_spark" a la tarea correcta. Para ello, consulta el sufijo de la tarea para saber qué cola utilizar. Pausa el vídeo ahora y nos vemos cuando hayas terminado.  Muy bien, espero que lo hayas conseguido.  Esta es la solución.  

<center><img src="https://i.postimg.cc/ZKsDXhvw/a892.png"></center>

Todas las tareas _ssd obtienen la cola "worker_ssd". Todas las tareas _cpu obtienen la cola "worker_cpu".  Y finalmente, la única tarea con _spark se pone con la cola "worker_spark".  Bien, no olvides guardar el archivo y ve a la terminal. Arranca los contenedores Docker ejecutando el mismo comando que hemos usado antes.  Enter.  

<center><img src="https://i.postimg.cc/zGLPrPn8/a893.png"></center>

Escribe "docker ps" para comprobar que todos los contenedores se están ejecutando.  Perfecto.  Ahora hay una cosa más que tenemos que cambiar.  **`Tenemos que definir qué cola debe consumir cada worker`**.  Como el comando para crear las colas se ha ejecutado para cada worker usando docker-compose, todos están escuchando las mismas colas, que no es lo que queremos. Para arreglar esto, ve al Flower Dashboard y refresca la página Ok.  Ahora haz click en el primer worker, luego en "Queues" y aquí vamos a mantener sólo dos colas, la por defecto y la asociada a este worker. 

<center><img src="https://i.postimg.cc/BbMwqrYQ/a894.png"></center>

El número aquí es el id del contenedor del worker. 

<center><img src="https://i.postimg.cc/sxDw0njF/a895.png"></center>

Recuérdalo, y desde el terminal podemos ver que corresponde al worker 3 que es el worker que tiene instalado Spark.  

<center><img src="https://i.postimg.cc/j2gZVz2D/a896.png"></center>

Así que sólo nos quedamos con la cola "worker_spark" “default” y eliminamos las demás. Ok, luego hacemos lo mismo para worker_1 y worker_2. 

<center><img src="https://i.postimg.cc/ZYVfyhHJ/a897.png"></center>

En "Dashboard", hacemos click en el segundo worker, luego en "Queues". Comprobamos el id del contenedor con los contenedores docker de la terminal.  Así que este es worker_2 que tiene más CPUs lo que significa que mantenemos la cola "worker_cpu" y “default”, y eliminamos las otras.  Hacemos lo mismo con el último worker, que es el que tiene un SSD para tareas intensivas de I/O.  Sólo mantenemos la cola "worker_ssd" “default”, y eliminamos las otras.  

Muy bien, de vuelta al "Dashboard", todo está configurado, podemos programar el DAG queue_dag desde la Airflow UI.  No se olvide de actualizar la página, y encienda el toggle del DAG.  Ahora actualice la página hasta que las primeras tareas en ejecución.  Ok. De vuelta al Flower dashboard, como puedes ver worker_1 ha ejecutado 3 tareas que son las 3 tareas de I/O del DAG.  

<center><img src="https://i.postimg.cc/28t22FgJ/a898.png"></center>

En realidad, podemos comprobar esto haciendo click en "Tasks".  Entonces en la caja de búsqueda si escribimos "_ssd", podemos ver que sólo el worker_1 ha ejecutado las tareas de I/O. 

<center><img src="https://i.postimg.cc/T3WQS56P/a899.png"></center>

Cambiamos _ssd por _cpu y las tareas intensivas de CPU han sido bien ejecutadas por el worker worker_2.  

<center><img src="https://i.postimg.cc/nLbTtVFG/a900.png"></center>

Finalmente, si cambiamos _cpu por _spark, obtenemos una única tarea ejecutada por el worker_3.

<center><img src="https://i.postimg.cc/8CR4k8sQ/a901.png"></center>

Así que todo ha funcionado como se esperaba.  Ahora te preguntarás ¿dónde se ha disparado la tarea 7?  Bueno, si escribes "task_7", obtenemos el siguiente worker.  

<center><img src="https://i.postimg.cc/tRS54NRT/a902.png"></center>

Por eso si vuelves al "Dashboard" y miras el worker 3 tenemos dos tareas procesadas en lugar de una correspondiente a las tareas t_6_spark y task_7.  

<center><img src="https://i.postimg.cc/658h24Th/a903.png"></center>

Muy bien, ahora estás siendo capaz de especializar tus nodos worker de acuerdo a los requerimientos de las tareas que quieres ejecutar. Como mejor práctica, incluso si todos los nodos de tu cluster tienen los mismos recursos, puede ser una buena idea especializar tus nodos worker.  Por ejemplo, puedes definir un nodo para que reciba tareas de carga de trabajo pesada, de forma que se procesen más rápidamente.  Las tareas ligeras no ocuparán recursos en ese nodo y se ejecutarán en los demás.  Las posibilidades son realmente ilimitadas, depende de tu arquitectura y de tu caso de uso. Antes de pasar al siguiente vídeo, vuelve al terminal, y detén los contenedores docker escribiendo "docker-compose -f docker-compose-Celery Executor.yml down".  Muy bien, espero que te haya gustado lo que has aprendido, vamos a tomar un breve descanso y nos vemos para el próximo video.   